# Determine the demographic breakdown of the participants

We want to know the most frequent categories of the participants, including all overlaps.

In [ ]:
import pandas as pd
import numpy as np
import itertools

In [ ]:
# this is a file that combines results from multiple surveys. 
# the challenge is going to be identifying the useful columns and what questions they belong to!
df = pd.read_spss('data/Consent_Pre_Post_MERGE.sav')
df

In [ ]:
cols = df.columns.values
print(cols)

In [ ]:
# I think these are the important columns
# They appear to come from the SP22_RQ2_Participant_ISTP.docx file
# missing faculty status question
useCols = [
    'primerole_march22', 'discipline_march22', 'institution_march22', 'gender_march22','firstgen0322', 'army',
    'institution1', 'institution8', 'institution9', 'institution10', 'institution11', 'institution12', 'institution13',
    'race_nativA_march22', 'race_asianA_march22',
    'race_africanA_march22', 'race_asianE_march22', 'race_latinx_march22',
    'race_MENA_march22', 'race_pi_march22', 'race_asianS_march22',
    'race_asianSE_march22', 'race_white_march22', 'race_multi_march22'
]

In [ ]:
usedf = df[useCols].fillna(0)
usedf

In [ ]:
def findRange(col):
    return [np.min(usedf[col].values), np.max(usedf[col].values)]

In [ ]:
for i,c in enumerate(useCols):
    print(c, findRange(c))

In [ ]:
# mapping for the answer from numbers to words from the SP22_RQ2_Participant_ISTP.docx file
roleMap = {
    1:'Faculty member, lecturer, instructor, or adjunct faculty',
    2:'Graduate student',
    3:'Posdoctoral scholar',
    4:'Staff member',
    5:'Other (role)'
}
disciplineMap = {
    1:'Agriculture and natural resource sciences',
    2:'Arts',
    3:'Biological and life sciences',
    4:'Business and management science',
    5:'Chemistry',
    6:'Computer, information, and technological sciences',
    7:'Earth, environmental, atmospheric, and ocean sciences',
    8:'Education',
    9:'Engineering',
    10:'Humanities',
    11:'Law',
    12:'Mathematics and Statistics',
    13:'Medical sciences',
    14:'Physical sciences',
    15:'Psychology',
    16:'Social, behavioral, and economic sciences (not including psychology)',
    17:'Other (discipline)'
}
institutionMap = {
    1:'Community college / 2-year institution',
    7:'Comprehensive or Regional University (e.g., smaller state school, schools that offer mostly bachelor or masters degrees)',
    8:'Liberal arts college',
    9:'Research University',
    10:'Technical college', 
    11:'Other (institution)'
}
genderMap = {
    1:'Gender queer or gender non-conforming', 
    8:'Man',
    9:'Nonbinary',
    10:'Transman',
    14:'Transwoman', 
    11:'Woman', 
    12:'I self-describe as (gender)',
    13:'I prefer not to respond (gender).'
}
firstgenMap = {1:'first gen'}
armyMap = {1:'veteran'}

# these are checkboxes so I will keep each individual column
insitutionTypeMap = {
    1:'Asian American and Pacific Islander Serving Institution (AAPISI)',
    8:'Hispanic Serving Institution (HSI)', 
    9:'Historically Black College and University (HBCU)', 
    10:'Predominantly White Institution (PWI)',
    11:'Tribal College/University',
    12:'Other Minority Serving Institution (MSI)',
    13:'I am not sure (institution)'
}
raceMap = {
    1:'Alaska Native, American Indian, Native American or Indigenous',
    14:'Asian American',
    15:'Black or African American',
    16:'East Asian',
    17:'Latina/o/x or Hispanic',
    18:'Middle Eastern or Northern African',
    19:'Pacific Islander',
    20:'South Asian',
    21:'Southeast Asian',
    22:'White',
    23:'Multiracial',
    24:'I self-describe as (race):',
    25:'I prefer not to respond (race).'
}
raceMap2 = {
    'nativA':'Alaska Native, American Indian, Native American or Indigenous',
    'asianA':'Asian American',
    'africanA':'Black or African American',
    'asianE':'East Asian',
    'latinx':'Latina/o/x or Hispanic',
    'MENA':'Middle Eastern or Northern African',
    'pi':'Pacific Islander',
    'asianS':'South Asian',
    'asianSE':'Southeast Asian',
    'white':'White',
    'multi':'Multiracial',
}

In [ ]:
i = 1
list(itertools.combinations(useCols, len(useCols)))

In [ ]:
# loop through all the columns and groupby all the other columns to get the overlaps
# if i == 0, then we find the overlaps of all columns
# if i > 0, then we ignore any columns < i, and replace their values with -1
# using method from : https://stackoverflow.com/questions/35268817/unique-combinations-of-values-in-selected-columns-in-pandas-data-frame-and-count

# I need to allow any 1 column to be anything, then any 2 columns, and so on.
n = 0
for i in np.arange(1, len(useCols)):
    # get all the combinations of i columns in useCols
    itr = list(itertools.combinations(useCols, i))
    print(i, len(itr))
    
    # these columns will be used in groupby while others will be anything
    for useColList in itr:
        g = usedf.groupby(list(useColList)).size().reset_index().rename(columns = {0:'count'})

        # add the missing column(s) as -1 (meaning could be anything)
        for cc in useCols:
            if (cc not in useColList):
                g.insert(0, cc, -1)
    
    
        # for the checkbox answers, I only care about those that have multiple rows with values > 0
        # so I will drop any of the groupdf rows that have only -1s and 0s with one non-zero value

        # remove the count
        count = g.pop('count')

        # get the trim condition
        condition = g.gt(0).sum(axis = 1).gt(1)
        gTrim = g[condition]

        # insert the count of >0 columns
        gTrim.insert(0, 'ngt0', gTrim.gt(0).sum(axis = 1))

        # put the count back in
        gTrim.insert(0, 'count', count[condition])



        if (n == 0):
            groupdf = gTrim
        else:
            groupdf = pd.concat([groupdf, gTrim])
            
        n += 1
        
    print(len(groupdf))
        
# for i,c in enumerate(useCols):
#     g = usedf.groupby(useCols[i:]).size().reset_index().rename(columns = {0:'count'})
    
#     # add the missing column(s) as -1 (meaning could be anything)
#     if (i > 0):
#         for cc in useCols[:i]:
#             g.insert(0, cc, -1)
    
#     # move the count column to the front
#     count = g.pop('count')
#     g.insert(0, 'count', count)

#     if (i == 0):
#         groupdf = g
#     else:
#         groupdf = pd.concat([groupdf, g])



# take only the rows with > 5 people in the group
groupdfTrim = groupdf.loc[groupdf['count'] > 5].reset_index(drop = True)

# sort
groupdfTrim.sort_values(by = 'count', ascending = False, inplace = True)

groupdfTrim



In [ ]:
# # for the checkbox answers, I only care about those that have multiple rows with values > 0
# # so I will drop any of the groupdf rows that have only -1s and 0s with one non-zero value

# # don't modify the original df
# tmp = groupdf.copy()

# # remove the count
# count = tmp.pop('count')

# # get the trim condition
# condition = tmp.gt(0).sum(axis = 1).gt(1)
# groupdfTrim = tmp[condition]

# # insert the count of >0 columns
# groupdfTrim.insert(0, 'ngt0', groupdfTrim.gt(0).sum(axis = 1))

# # put the count back in
# groupdfTrim.insert(0, 'count', count[condition])

# # take only the rows with > 5 people in the group
# groupdfTrim = groupdfTrim.loc[groupdfTrim['count'] > 5].reset_index(drop = True)

# groupdfTrim


In [ ]:
# replace numbers with values
replacements = {
    'primerole_march22':roleMap,
    'discipline_march22':disciplineMap,
    'institution_march22':institutionMap,
    'gender_march22':genderMap,
    'firstgen0322':firstgenMap,
    'army':armyMap
}

for key, value in insitutionTypeMap.items():
    col = 'institution' + str(key)
    replacements[col] = {1:value}
    
for key, value in raceMap2.items():
    col = 'race_' + key + '_march22'
    replacements[col] = {1:value}

groupdfTrimHuman = groupdfTrim.replace(replacements)

groupdfTrimHuman


In [ ]:
# print with human readable results
counts = []
groups = []
for i,row in groupdfTrim.iterrows():
    foo = row.copy()
    foo.pop('count')
    foo.pop('ngt0')
    
    bar = groupdfTrimHuman.iloc[i].copy()
    bar.pop('count')
    bar.pop('ngt0')
    
    # find the columns with values > 0
    condition = foo.gt(0)
    
    counts.append(int(row['count']))
    groups.append('; '.join(bar[condition].values))
    
    #print(f'{c:.0f} : {foo[condition].to_dict()}')

# combine this into a dataframe so that I can easily drop the duplicates
# (duplicates arise when I add a column to groupby and then it gets selected for a 0)
outdf = pd.DataFrame({'count':counts, 'group':groups}).drop_duplicates(subset = 'group', keep = 'first')
outdf

In [ ]:
outdf.to_csv('data/demographicsGroups.csv', index = False)

## Scratch below

In [ ]:
# this file does not have enough demographics info
df = pd.read_spss('data/ParticipantProfile.sav')
df

In [ ]:
cols = df.columns.values
print(cols)

In [ ]:
# looks like this data is included in the other file and not any more helpful to distinguish which questions is which
df = pd.read_spss('data/Pre_Survey_Oct21.sav')
df

In [ ]:
cols = df.columns.values
print(cols)